# Instability Detection and Characterization

This tutorial shows how to implement instability ("drift") detection and characterization, on time-stamped data. This data can be from *any* quantum circuits, on *any* number of qubits. For example, possible experiments include suitably time-ordered GST, RPE, Ramsey or RB experiments.


Not time-resolved tomography

This notebook is an introduction to these tools, and it will be either augmented with further notebooks, or updated to be more comprehensive, at a later date.

In [1]:
from __future__ import print_function

# Importing the drift module is essential
from pygsti.extras import drift

# Importing all of pyGSTi is optional, but often useful.
import pygsti

Welcome to pygsti version 0.9.7!
There have been some major changes between this version and 0.9.6 - ones that break backward compatibility.  If you're trying to run an old script and nothing works, DON'T PANIC; we've tried to make the transition easy.  More often then not, you can just run `pyGSTi/scripts/upgrade2v0.9.7.py` on your old script or notebook files and you'll be up and running again.  For more information, see the pyGSTi FAQ.ipynb.

If this warning annoys you run:
  `export PYGSTI_BACKCOMPAT_WARNING=0` from the command line or
  `import os; os.environ['PYGSTI_BACKCOMPAT_WARNING'] = '0'` in a script or
     notebook *before* importing pygsti and the the madness will stop.


## Quick and Easy Analysis
First we import some *time-stamped* data. For more information on the mechanics of using time-stamped `DataSets` in `pyGSTi` see the [TimestampedDataSets](../objects/advanced/TimestampedDataSets.ipynb) tutorial. Later we'll discuss the types of time-stamped data that this analysis can deal with, i.e., the type of experiment that we can analyze the output of.

The data we're importing is from long-sequence GST on $G_i$, $G_x$, and $G_y$ with ...

In [ ]:
ds = pygsti.io.load_tddataset("../tutorial_files/timeseries_data.txt")

Then we hand this data to `drift.do_stability_analysis()`.

In [ ]:
# This'll take 5 - 10 minutes.
#results = drift.do_stability_analysis(ds)

In [2]:
import pickle 
#with open('results.pkl','wb') as f:
#    pickle.dump(results, f)
with open('results.pkl','rb') as f:
    results = pickle.load(f)

## Inspecting the results

Everything has been calculated, and we can now look at the results. If we print the results, it will tell us whether instability was detected. If no instability was detected, then there is little else to do.

But note that whether instability is detected is obviously a function of the amount of data, the size of any instability, and the circuits run. Not detecting instability is **not** a reliable certificate of stability unless ....

In [ ]:
print(results)

In [ ]:
# Create a workspace to show plots
w = pygsti.report.Workspace()
w.init_notebook_mode(connected=False, autodisplay=True) 

### 1. Instability Detection Results : Power Spectra and the Frequencies of Instabilities

These spectra *should* be flat - up to statistical flucations, due to finite-sampling noise, around the mean noise level - if there is no drift. There are a range of power spectra that we can plot, but the most useful for an overview of the data is the "global power spectrum", obtained from averaging power spectra calculated from the individual data for each of the different operation sequences (again, details on exactly what this is are given later). This is plotted below. If there are peaks above the significance threshold, this power spectra provides statistically significant evidence of drift.

In [ ]:
w.PowerSpectraPlot(results)

We would probably likely like to know the frequencies of the drift. This information can be extracted from the results object as shown below. All frequencies will be in Hz if the timestamps have been provided in seconds (again, details later). Note that these are the frequencies in the drifting outcome probabilities -- they are *not* directly the frequencies of drift in, say, a Hamiltonian parameter. However, they are closely related to those frequencies.

In [ ]:
results.get_instability_frequencies()

In [ ]:
circuits = {L: pygsti.obj.Circuit(None,stringrep='Gx(Gi)^'+str(L)+'Gx') for L in [1,2,4,16,64,128,256]}
w.PowerSpectraPlot(results, {'circuit':circuits}, showlegend=True)

We can return a dictionary where the keys are circuits, and the values are the frequencies found to be significant for that circuit, i.e., they are almost certaintly components in the probability trajectory for that circuit. 

In [ ]:
unstablecircuits = results.get_unstable_circuits()
# We only display the first 10 circuits and frequencies, as there are a lot of them!
for ind, (circuit, freqs) in enumerate(unstablecircuits.items()):
    if ind < 10: print(circuit.str, freqs)

### 2. Instability Characterization Results : Probability Trajectories

We can look at the estimated probability trajectory for any circuit of interest (or a selection of circuits, if we instead hand the plotting function a dictionary of circuits).

In [ ]:
circuit = pygsti.obj.Circuit(None, stringrep= 'Gx(Gi)^256GxGxGx')
w.ProbTrajectoriesPlot(results, circuit, ('1',))

### 3. Further plotting for data from structured circuits (e.g., GST circuits)

If the data is from GST experiments, or anything with a GST-like structure of germs and fudicials, we can create some extra plots...

a box-plot which shows the maximum power in the spectrum for each sequence. This maximum power is a reasonable proxy for comparing how "drifty" the data from the different sequences appears to be. But note that the maximum power should *not* be used to directly compare the level of drift in two different datasets with different parameters, particularly if the number of timestamps is different - because this maximum power will increase with more data, for a fixed level of drift. More on this at a later date.

In the plot below we see that the amount of drift appears to be increasing with sequence length, as would be expected with gate drift. Without performing a detailed analysis, by eye it is clear that the $G_i$ gate is the most drifty, that the $G_x$ gate has some drift, and that the data looks consistent with a drift-free $G_y$ gate.

In [3]:
from pygsti.construction import std1Q_XYI # The model used with the GST data we imported

# This manually specifies the germ and fiducial structure for the imported data.
fiducial_strs = ['{}','Gx','Gy','GxGx','GxGxGx','GyGyGy']
germ_strs = ['Gi','Gx','Gy','GxGy','GxGyGi','GxGiGy','GxGiGi','GyGiGi','GxGxGiGy','GxGyGyGi','GxGxGyGxGyGy']
log2maxL = 9 # log2 of the maximum germ power

# Below we use the maxlength, germ and fuducial lists to create the GST structures needed for box plots.
fiducials = [pygsti.objects.Circuit(None,stringrep=fs) for fs in fiducial_strs]
germs = [pygsti.objects.Circuit(None,stringrep=mdl) for mdl in germ_strs]
max_lengths = [2**i for i in range(0,log2maxL)]
gssList = pygsti.construction.make_lsgst_structs(std1Q_XYI.gates, fiducials, fiducials, germs, max_lengths) 

In [ ]:
# Create a boxplot of the maximum power in the power spectra for each sequence.
w.ColorBoxPlot('driftdetector', gssList[-1], None, None, stabilityanalyzer=(results,None,None))

In [ ]:
# Create a boxplot of the maximum power in the power spectra for each sequence.
w.ColorBoxPlot('driftsize', gssList[-1], None, None, stabilityanalyzer=(results, None, None))

In [ ]:
germ = 'Gi'
outcome = ('1',)
# Use the drop-down menu to pick a fiducial pair (it initializes including all of them),
# and use the slider to zoom in 
w.GermProbTrajectoriesPlot(results, gssList[-1], germ, outcome, minL=8)

In [4]:
drift.driftreport.create_drift_report(results, gssList[-1], "../tutorial_files/DriftReport",
                                      title="Example Drift Report", verbosity=10)

*** Creating workspace ***
*** Generating switchboard ***
*** Generating tables ***
  sampleTable                                   took 0.000161 seconds
*** Generating plots ***
  driftdetectorColorBoxPlot                     took 107.212034 seconds
  driftsizeColorBoxPlot                         took 0.402706 seconds
  GermFiducialProbTrajectoriesPlot              took 6.1e-05 seconds


AttributeError: 'numpy.ndarray' object has no attribute 'switchedCompute'

In [ ]:
gss = gssList[-1]

In [ ]:
gss.

You can now open the file [../tutorial_files/DriftReport/main.html](../tutorial_files/DriftReport/main.html) in your browser (Firefox works best) to view the report.

# NOTHING BELOW HERE WORKS

**Constructing estimates of the drifting probabilities**

The analysis creates estimates of the time-dependent probability, $p(t)$, to obtain a given outcome, for each sequence. Below, we will explain how to access and plot all of these estimates. Here, we demonstrate how to plot the estimated $p(t)$ that the analysis concludes is "the most drifty" (the $p(t)$ where the estimated drift has the highest power). Obviously, the more this function oscillates, the more drifty the data for this sequence appears to be.

In [ ]:
#results_gst.plot_most_drifty_probability(plot_data=True)

It can also be informative to display multiple reconstructions on a single plot. Below we plot the reconstructions for the $G_i$ and $G_x$ germs, both with varying germ power and fixed fudicials.

In [ ]:
# Pick a list of sequence labels (here circuits) or indices to plot the estimated p(t) for
gstrs = [pygsti.objects.Circuit(None,stringrep='Gx(Gi)^'+str(2**l)+'Gy') for l in range(4,10)]
# Hand this list to the plotting function
#results_gst.plot_probability_trajectory_estimates(gstrs)

In [ ]:
# Pick a list of sequence labels (here circuits) or indices to plot the estimated p(t) for
gstrs = [pygsti.objects.Circuit(None,stringrep='(Gx)^'+str(2**l)) for l in range(4,10)]
# Hand this list to the plotting function
#results_gst.plot_multi_estimated_probabilities(gstrs,loc='upper right')

**More details please?**

Having provided an overview of the tools in the `drift` module, we now give a more detailed introduction to:
1. The types of data that can be analyzed, and the format they must be imported in to use the `drift` module.
2. The analysis methods that are used inside the `drift.do_basic_drift_characterization()`.
3. How to access the various outputs returned by the analysis, and some details on how to interpret them. 

## Input data types and formats
#### What types of data can be analyzed?

 The methods in this notebook can be used on data from any experiment that satisfies the following criteria: 

- The experiments consist of $S \geq 1$ different circuits, or sequences, each with $M \geq 2$ possible outcomes with $M$ the same for all $S$ circuits.


- Sequence $s$ is repeated, and one of the $M$ outcomes is recorded, $N$ times during each of $T$ different time-intervals, with $N$ and $T$ the same for all sequences $s$. Let $\tau_{s,1},\tau_{s,2},\dots,\tau_{s,T}$ denote times associated with these time-intervals (e.g., the start time or mid-point of each interval).


- The time-gap between consecutive sets of $N$ repeats of a sequence is approximately constant, and approximately independent of sequence. That is $t_{s,gap} \approx \tau_{s,t+1} - \tau_{s,t}$ for all $s,t$ and some constant $t_{s,gap}$ (the time gap is constant for each sequence), and $t_{gap} \equiv t_{s,gap} \approx t_{s',gap}$ for all $s,s'$ (the time gap is the same for every sequence).

#### What type of circuits can the data come from?

Many characterization routines, including GST, RPE, restricted sets of GST sequences (e.g., Ramsey sequences) or RB, satisfy the above criteria *if* a suitable time-ordering for the repeats of all of the sequences is chosen.

An example is a full set of GST sequences, with sequence 1 repeated $N=5$ times, then sequence 2 repeated $N=5$ times, etc, with this entire procedure looped through $T = 500$ times. As long as each circuit follows on from the following circuit with a roughly constant time gap, then -- even if the different circuits take varying times to implement -- the time-gaps between consecutive sets of $N$ repeats of each circuit are all the same.

To obtain best performance, it is preferable to minimize $N$ and maximize $T$ for the same fixed $N \times T$ (this will provide better detection for high frequency drift).




## Example 1 : Single sequence data
We demonstrate this on single-sequence data, for a single entity, with two-outcome measurements. This is the circumstance under which the analysis function can be given data in a 1D array, and so we will use this method below. We now demonstrate the method with simulated data for a drifting and drift free probability.

Below we use a drifting probability of
$$ p(t) = 0.5 + 0.2 \cos(0.1 t).,$$
with integer $t$ in the range $0,1,2,\dots,T$.
This is similar in form to the drifting probability that would be obtained with certain sorts of Ramsey experiment with drifting $\sigma_z$ over-rotation angle.

In [ ]:
# Imports for generating fake data to demonstrate the methods.
from numpy.random import binomial
from numpy.random import multinomial

In [ ]:
# N = 5 # Counts per timestep
# T = 100 # Number of timesteps

# # The confidence of the statistical tests. Here we set it to 0.999, which means that
# # if we detect drift we are 0.999 confident that we haven't incorrectly rejected the
# # initial hypothesis of no drift.
# confidence = 0.999

# # A drifting probability to obtain the measurement outcome with index 1 (out of [0,1])
# def pt_drift(t): return 0.5+0.2*np.cos(0.1*t)

# # A drift-free probability to obtain the measurement outcome with index 1 (out of [0,1])
# def pt_nodrift(t): return 0.5

# # If we want the sequence to have a label, we define a list for this (here, a list of length 1).
# # The labels can, but need not be, pyGSTi OpString objects.
# sequences = [pygsti.objects.OpString(None,'Gx(Gi)^64Gx'),]

# # If we want the outcomes to have labels, we define a list for this.
# outcomes = ['0','1']

# # Let's create some fake data by sampling from these p(t) at integer times. Here we have
# # created a 1D array, but we could have instead created a 1 x 1 x 1 x T array.
# data_1seq_drift = np.array([binomial(N,pt_drift(t)) for t in range(0,T)])
# data_1seq_nodrift = np.array([binomial(N,pt_nodrift(t)) for t in range(0,T)])

# # If we want frequencies in Hertz, we need to specify the timestep in seconds. If this isn't
# # specified, the frequencies are given in 1/timestep with timestep defaulting to 1.
# timestep = 1e-5

# # We hand these 1D arrays to the analysis function, along with the number of counts, and other
# # optional information
# results_1seq_drift = drift.do_basic_drift_characterization(data_1seq_drift, counts=N, outcomes=outcomes,
#                                                            confidence=confidence, timestep=timestep, 
#                                                            indices_to_sequences=sequences)
# results_1seq_nodrift = drift.do_basic_drift_characterization(data_1seq_nodrift, counts=N, outcomes=outcomes, 
#                                                              confidence=confidence, timestep=timestep, 
#                                                              indices_to_sequences=sequences)                                                    

We can now compare the global power spectrum obtained when there is drift, and when there is not drift. Except in highly unusual cases (1 in 1000) there will be no drift detected when there is indeed no drift. 

In [ ]:
# results_1seq_drift.plot_power_spectrum()

In [ ]:
# results_1seq_nodrift.plot_power_spectrum()

We can look to see what the p-value associated with the largest peak in the power spectrum is. 

In [ ]:
# print(results_1seq_drift.global_pvalue)
# print(results_1seq_nodrift.global_pvalue)

The analysis function performs:
1. an analysis on a per-sequence, per-entity, per-outcome basis (properties starting with `pspepo`), an analysis on a per-sequence, per-entity, outcome-averaged basis (properties starting with `pspe`), 
2. an analysis on a per-sequence, entity-averaged, outcome-averaged basis (properties starting with `ps`), 
3. an analysis on a sequence-averaged, per-entity, outcome-averaged basis (properties starting with `pe`),
4. a global analysis on a sequence-averaged, entity-averaged, outcome-averaged basis (properties starting with `global`)

That is, it inspects all the power spectra, after the relevant averaging has been performed, for evidence of drift (e.g., for case 1 there are $S \times E \times M \times T$ spectra).

In the example here there is a single sequence, a single entity, and two-outcome measurements. Hence, in this case all of the analyzes are exactly equivalent, as is clear by noting that all of the power spectra are the same:

In [ ]:
# # The power spectrum obtained after averaging over everthing
# print(results_1seq_drift.global_power_spectrum[:4])
# # The power spectrum obtained after averaging over everthing except sequence label
# print(results_1seq_drift.ps_power_spectrum[0,:4])
# # The power spectrum obtained after averaging over everthing except entity label
# print(results_1seq_drift.pe_power_spectrum[0,:4])
# # The power spectrum obtained after averaging over everthing except sequene and entity label
# print(results_1seq_drift.pspe_power_spectrum[0,0,:4])
# # The two power spectra obtained after averaging over nothing
# print(results_1seq_drift.pspepo_power_spectrum[0,0,0,:4])
# print(results_1seq_drift.pspepo_power_spectrum[0,0,1,:4])

As we can see above, the power spectra for the two different measurement outcomes are identical. This is because the Fourier modes for all of the measurement outcomes must sum to zero. Hence, there are only $M-1$ independent power spectra, which here is 1.

As already demonstrated above, the analysis function also creates an estimate for the drift probability for each measurement outcome (and for each sequence, and each entity, but here there is only one of each of these). This estimate is created in the following way:

1. Inspect the relevant power spectrum (the power spectrum for the sequence, entity, and measurement outcome under consideration).
2. Take the modes obtained from the data, and set all modes with power below the "significance threshold" to zero. This significance threshold is generally adjusted for the number of sequences and entities considered, which is not relevant in this case but is discussed below.
3. Keep all modes with power above the significance threshold and invert the Fourier transform.
4. This is the estimate of $p(t)$, up to some final adjustments to guarantee that $p(t)$ is within $[0,1]$.

Because we have created our data from a known underlying $p(t)$ we can compare our estimate of $p(t)$ with the true $p(t)$. We do this below.

In [ ]:
# # Lets create an array of the true probability. This needs to be
# # of dimension S x E x M x T
# parray_1seq = np.zeros((1,1,2,T),float)
# parray_1seq[0,0,0,:] = np.array([pt_drift(t) for t in range(0,T)])
# parray_1seq[0,0,1,:] = 1 - parray_1seq[0,0,0,:]

# # The measurement outcome index we want to look at (here the esimated p(t) 
# # for one index is just 1 - the p(t) for the other index, because we are
# # looking at a two-outcome measurement).
# outcome = 1

# # If we hand the parray to the plotting function, it will also plot
# # the true probability alongside our estimate from the data
# results_1seq_drift.plot_estimated_probability(sequence=0,outcome=outcome,parray=parray_1seq,plot_data=True)

## Example 2 : Single-sequence multi-qubit data
Single sequence data with two measurement outcomes is (a) often not will be obtained in experiments, and (b) is not sufficient to demonstrate all of the methods contained within the analysis function, or to understand all of the ouput. Hence, we now consider an example with a single-sequence and 4 measurement outcomes, which could represent a two-qubit single-sequence experiment.

Let us consider the case of 4 possible measurement outcomes:

In [ ]:
# We only explicitly name them for labelling purposes.
# outcomes = ['00','01','10','11']

This could correspond to computational basis measurements of two qubits. Now let us assume that the drifting $p(t)$ for obtaining each measurement outcome factorizes into probabilities for outcomes 0 and 1 for each qubit (e.g., because it is parallel single-qubit experiments and there is no crosstalk). Specifically, take the first qubit to have drifting probability for outcome 0 of
$$ p_{0}(t) = 0.5+0.05 \cos(0.08t),$$
and the second qubit to have drifting probability for outcome 0 of
$$ p_{0}(t) = 0.5+0.05 \cos(0.2t),$$
with $t \in [0,1,\dots, T]$.

Let's create some data from these drifting probabilities

In [ ]:
# N = 10 # Counts per timestep
# T = 1000 # Number of timesteps

# # The drifting probabilities for the 4 outcomes
# def pt00(t): return (0.5+0.07*np.cos(0.08*t))*(0.5+0.08*np.cos(0.2*t))
# def pt01(t): return (0.5+0.07*np.cos(0.08*t))*(0.5-0.08*np.cos(0.2*t))
# def pt10(t): return (0.5-0.07*np.cos(0.08*t))*(0.5+0.08*np.cos(0.2*t))
# def pt11(t): return (0.5-0.07*np.cos(0.08*t))*(0.5-0.08*np.cos(0.2*t))

# # Because of the type of input (>2 measurement outcomes), we must record the
# # data in a 4D array (even though some of the dimensions are trivial)
# data_multiqubit = np.zeros((1,1,4,T),float)

# # Generate data from these p(t)
# for t in range(0,T):
#     data_multiqubit[0,0,:,t] = multinomial(N,[pt00(t),pt01(t),pt10(t),pt11(t)])

#### Example 2.1 : A simple analysis of the full dataset
The simplest way to analyze this data is just to hand it to the drift characterization method, as before (note that now we don't need to specify the counts number, as the input is a 4D array). We do this below.

In [ ]:
# results_multiqubit_full = drift.do_basic_drift_characterization(data_multiqubit,outcomes=outcomes,confidence=0.99)

By looking at the global power spectrum, we can clearly see that there is drift.

In [ ]:
# results_multiqubit_full.plot_power_spectrum()

As always, we can also return the frequencies, found from inspecting the global power spectrum. As we have not specified a timestep, these are integers between $1$ and $T-1$.

In [ ]:
# print(results_multiqubit_full.global_drift_frequencies)

Because there are multiple measurement outcomes, we can look at the power spectra for each of these, and also we can look at the drift frequencies found from just analyzing this spectrum. One example of such a power spectrum is given below

In [ ]:
# outcome = '00'
# results_multiqubit_full.plot_power_spectrum(sequence=0,entity=0,outcome=outcome)

The drift frequencies found via an independent analysis of each of these spectra. Generally, these need not contain the same frequencies as the global analysis. In this example, because all of the frequencies appear in all of the spectra, the global analysis is more sensitive (reduced noise without reducing signal strength). 

In [ ]:
# print(results_multiqubit_full.pspepo_drift_frequencies[0,0,0])
# print(results_multiqubit_full.pspepo_drift_frequencies[0,0,1])
# print(results_multiqubit_full.pspepo_drift_frequencies[0,0,2])
# print(results_multiqubit_full.pspepo_drift_frequencies[0,0,3])

As always, we can also look at the reconstruction for a given drifting probability. An example is shown below, where again we construct the true underlying probability with which to compare it.

In [ ]:
# # Creates an array of the true probability.
# parray_multiqubit_full = np.zeros((1,1,4,T),float)
# parray_multiqubit_full[0,0,0,:] = np.array([pt00(t) for t in range(0,T)])
# parray_multiqubit_full[0,0,1,:] = np.array([pt01(t) for t in range(0,T)])
# parray_multiqubit_full[0,0,2,:] = np.array([pt10(t) for t in range(0,T)])
# parray_multiqubit_full[0,0,3,:] = np.array([pt11(t) for t in range(0,T)])

# results_multiqubit_full.plot_estimated_probability(sequence=0,outcome=1, plot_data=True,
#                                                    parray=parray_multiqubit_full)

#### Example 2.2 : Marginalizing over qubits

The above analysis looks for drift in the $2^2$ measurements outcomes of a two-qubit circuit. This would be problematic for many qubits, as for $Q$ qubits there is $2^Q$ possible measurement outcomes, and the probability of any one outcome being observed will - for many circuits - converge to zero as $Q$ increases. Then, even for very large $T$, there could be, e.g., at most 2 counts for any one measurement outcome, and the power spectra obtained will be useless.

However, it is likely in many circumstances that any drift will show up in the marginalized probabilities for obtaining 0 or 1 of each qubit. E.g., in the example above this is precisely what is drifting. We can implement an analysis like this by specifying that the analysis function marginalizes.

To do this in an automated way, we specify ` marginalize = 'std'`. This then assumes that the data is such that the 0th indexed outcomes is the bitstring 0...0000, the 1st indexed outcome is the bitstring 0...0001, etc. It then associates the first qubit with entity 0 (where first qubit has its measurement outcome recorded as the first bit in the string), the second qubit with entity 1 and so on.

Note that, when using this method, the input data array grows exponentially in the number of qubits. As such, pre-marginalizing will be preferable for a large number of qubits, in which case the data for different qubits should be stored with different "entity" indices.

Below we demonstrate this method on the same data as analyzed above

In [ ]:
# results_multiqubit_marg = drift.do_basic_drift_characterization(data_multiqubit, outcomes=outcomes, 
#                                                                 marginalize = 'std', confidence=0.99)

As we can see from the returned data shape, the data has been split into separate sets for 2 different entities, each have two possible measurement outcomes.

In [ ]:
# np.shape(results_multiqubit_marg.data)

As always, we can plot the global power spectrum (averaged over everything, including entity, i.e., qubit).

In [ ]:
# results_multiqubit_marg.plot_power_spectrum()

However, now we can look at the power spectrum for each individual qubit. This then shows us that each qubit has a different drift frequency. Something that was not obvious from the analysis implemented without marginalization. Moreover, these spectra show higher peaks (they are approximately twice as high), because -- earlier -- when we constructed the global power spectrum we were averaging pure noise with "signal" at both drift frequencies.

In [ ]:
# results_multiqubit_marg.plot_power_spectrum(sequence=0,entity=1)

In [ ]:
# results_multiqubit_marg.plot_power_spectrum(sequence=0,entity=0)

With this analysis we can isolate the drift frequencies of each qubit

In [ ]:
# # Drift frequencies for the first qubit
# print(results_multiqubit_marg.pe_drift_frequencies[0])
# # Drift frequencies for the second qubit
# print(results_multiqubit_marg.pe_drift_frequencies[1])

Finally, we can again plot the estimated drift probabilities. Again, these are more informative that what was obtained without marginalization. In particular, the oscillations are noticably stronger, as they are each the sum of two of the drifting probabilities from the "raw" unmarginalized case.

In [ ]:
# # Creates an array of the true probability.
# parray_multiqubit_marg = np.zeros((1,2,2,T),float)
# parray_multiqubit_marg[0,0,0,:] = np.array([pt00(t)+pt01(t) for t in range(0,T)])
# parray_multiqubit_marg[0,0,1,:] = np.array([pt10(t)+pt11(t) for t in range(0,T)])
# parray_multiqubit_marg[0,1,0,:] = np.array([pt00(t)+pt10(t) for t in range(0,T)])
# parray_multiqubit_marg[0,1,1,:] = np.array([pt01(t)+pt11(t) for t in range(0,T)])

# results_multiqubit_marg.plot_estimated_probability(sequence=0,entity=0,outcome=0,parray=parray_multiqubit_marg)

#### Example 2.3 : Marginalizing over qubits loses information and can miss detectable drift
Marginalizing involves throwing away information about correlations between measurement outcomes. As such, it is insensitive to certain types of drift. In particular, if only the correlations between measurement outcomes are drifting. This is perhaps a rather contrived situation, but it is possible. For example, consider the drifting probabilities:
$$ p_{00}(t) = 0.25-0.05 \cos(0.05*t) $$
$$ p_{01}(t) = 0.25+0.05 \cos(0.05*t) $$
$$ p_{10}(t) = 0.25+0.05 \cos(0.05*t) $$
$$ p_{11}(t) = 0.25-0.05 \cos(0.05*t) $$
In this case, as demonstrated below, the analysis of the full data clearly demonstrates drift, yet the analysis of the marginalized data does not.

In [ ]:
# N = 10 # Counts per timestep
# T = 1000 # Number of timesteps

# outcomes = ['00','01','10','11']

# def pt_correlated00(t): return 0.25-0.05*np.cos(0.05*t)
# def pt_correlated01(t): return 0.25+0.05*np.cos(0.05*t)
# def pt_correlated10(t): return 0.25+0.05*np.cos(0.05*t)
# def pt_correlated11(t): return 0.25-0.05*np.cos(0.05*t)

# data_1seq_multiqubit = np.zeros((1,1,4,T),float)
# for t in range(0,T):
#     pvec = [pt_correlated00(t),pt_correlated01(t),pt_correlated10(t),pt_correlated11(t)]
#     data_1seq_multiqubit[0,0,:,t] = multinomial(N,pvec)
        
# results_correlatedrift_marg = drift.do_basic_drift_characterization(data_1seq_multiqubit,
#                                                              outcomes=outcomes, marginalize = 'std')

# results_correlatedrift_full = drift.do_basic_drift_characterization(data_1seq_multiqubit, 
#                                                              outcomes=outcomes, marginalize = 'none')

In [ ]:
# results_correlatedrift_marg.plot_power_spectrum()

In [ ]:
# results_correlatedrift_full.plot_power_spectrum()

What this demonstrates is that it is important to carefully decide how to analyze data from a given experiment, with physical motivations for the analysis used.